<a href="https://colab.research.google.com/github/Francisco0a/paneldata1/blob/Notebook/Panel_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import pandas as pd
import numpy as np
drive.mount('/content/drive')
pd.set_option('display.max_columns', None)
# Uploading the filtered dataset with the department code 69 as a parquet file
####1.Transform if necessary the database in the right format.
####2.	Select your “département.” in the code.

# filtered_df = diane_data[diane_data['départementcode'] == 69]

file_path = '/content/drive/My Drive/filtered_df.parquet'
filtered_df = pd.read_parquet(file_path)
for col_name, dtype in filtered_df.dtypes.items():
    print(f'{col_name}: {dtype}')
# Basic descriptive statistics
print(filtered_df.describe())
# Frequency of observations per year
print(filtered_df['year'].value_counts())
# Frequency of observations per company
print(filtered_df['siren'].value_counts())
distribution = filtered_df['départementcode'].value_counts()
print(distribution)
# Check if the panel is balanced
unique_years_per_siren = filtered_df.groupby('siren')['year'].nunique()
max_years = unique_years_per_siren.max()
min_years = unique_years_per_siren.min()

if max_years == min_years:
    print(f"The panel is balanced with each entity having {max_years} time periods.")
else:
    print(f"The panel is unbalanced. The maximum number of time periods for an entity is {max_years}, and the minimum is {min_years}.")


Mounted at /content/drive
siren: float64
year: int16
companyname: object
captem_fa: float64
wc_currentas: float64
currentratio: float64
sirenID: object
nafrev2primarycodecode: object
lastavailableyear: object
regioncode: float64
region: object
numberofbranches: float32
noofshareholders: int16
nacerev2primarycodecode: float64
bankname: object
solvencyratio: float64
totalassets: float64
totalfixedassets: float64
ÿþmarquée: float64
nomdelentreprise: object
nafrév2codeprincipalcode: object
codecommune5chiffres: object
datededébutdactivité: object
effectifmoyendupersonnel: float64
brevet: float64
départementcode: float64
nacerév2codeprincipalcode: float64
HT: float32
MHT: float32
MLT: float32
LT: float32
KIS: float32
HTKIS: float32
FKIS: float32
OKIS: float32
KIS_all: float32
HT_all: float32
man: float32
trade: float32
elec_eau: float32
agr: float32
trans: float32
ict: float32
real: float32
cons: float32
services: float32
roa: float64
deprtcode: float32
currentas: float64
creditors: float64

In [2]:
###3.	Compute the ratio (Cash+marketable securities)/Totalassets) for numerator >=0 and denominator >=1 and its lag.
###Check that, for each firm, the first data should be missing.
# Step 1: Compute the ratio where conditions are met
filtered_df['ratio'] = np.where(
    (filtered_df['cashNbank'] + filtered_df['marketable_sec'] >= 0) & (filtered_df['totalassets'] >= 1),
    (filtered_df['cashNbank'] + filtered_df['marketable_sec']) / filtered_df['totalassets'],
    np.nan
)

# Step 2: For each firm, calculate the lag of the computed ratio
filtered_df['ratio_lag'] = filtered_df.groupby('siren')['ratio'].shift(1)

# Verify that the first data point for each firm in 'ratio_lag' is missing
print(filtered_df[['siren', 'year', 'ratio', 'ratio_lag']].head())

# Optional: You can check that the first observation for each 'siren' is NaN for 'ratio_lag'
first_lag_check = filtered_df.groupby('siren')['ratio_lag'].apply(lambda x: x.isnull().iloc[0])
print("First data for each firm is missing in 'ratio_lag':", first_lag_check.all())
ratio = filtered_df['ratio_lag'].value_counts()
print(ratio)

         siren  year  ratio  ratio_lag
115  5720651.0  2014    0.0        NaN
116  5720651.0  2015    0.0        0.0
117  5720651.0  2016    0.0        0.0
118  5720651.0  2017    NaN        0.0
119  5720651.0  2018    NaN        NaN
First data for each firm is missing in 'ratio_lag': True
ratio_lag
0.000000    6200
0.500000     304
0.333333     232
0.200000     214
0.250000     213
            ... 
0.222044       1
0.867052       1
0.598039       1
0.082127       1
0.282031       1
Name: count, Length: 51942, dtype: int64


In [3]:
# Generate dummy variables for each year
filtered_df['Dum14'] = (filtered_df['year'] == 2014).astype(int)
filtered_df['Dum15'] = (filtered_df['year'] == 2015).astype(int)
filtered_df['Dum16'] = (filtered_df['year'] == 2016).astype(int)
filtered_df['Dum17'] = (filtered_df['year'] == 2017).astype(int)
filtered_df['Dum18'] = (filtered_df['year'] == 2018).astype(int)


In [4]:
filtered_df.head(20)

,siren,year,companyname,captem_fa,wc_currentas,currentratio,sirenID,nafrev2primarycodecode,lastavailableyear,regioncode,region,numberofbranches,noofshareholders,nacerev2primarycodecode,bankname,solvencyratio,totalassets,totalfixedassets,ÿþmarquée,nomdelentreprise,nafrév2codeprincipalcode,codecommune5chiffres,datededébutdactivité,effectifmoyendupersonnel,brevet,départementcode,nacerév2codeprincipalcode,HT,MHT,MLT,LT,KIS,HTKIS,FKIS,OKIS,KIS_all,HT_all,man,trade,elec_eau,agr,trans,ict,real,cons,services,roa,deprtcode,currentas,creditors,financialdebtequity,netsales,dateofincorporation,sharecapital,convertibleloans,otherdebentureloans,bankborrowings,otherborrowings,typeofbusiness,NetIntangible,year_incorporation,tradedebts,marketable_sec,cashNbank,land,buildings,plantAequip,other_tangible,nettangible,fincharges,purch_good,purch_raw,wage_salaries,PropPlEq_inPRG,prepay_tangibleAs,conso_insvestm,customerprepay,taxes,debttosupplier_tangAs,otherdebts,totalDebt,InterestR,totalrevenues,totalexpenses,rawmaterials,workinprogr,ServINprogr,semifinNfinGoods,goodforsales,netincome,inventory,goodwill,othIntangAs,ratio,ratio_lag,Dum14,Dum15,Dum16,Dum17,Dum18
115,5720651.0,2014,SA DES ETS RENE DUFOUR,1.46,1.00,NaN,5720651,6820B,31/12/2016,84.0,Auvergne-Rhône-Alpes,0.0,1,6820.0,,91.50,128.0,259.0,4664.0,SA DES ETS RENE DUFOUR,6820B,69382,01/01/2014,NaN,0.0,69.0,6820.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,69.0,120.0,0.0,9.29,0.0,01/01/1957,67.0,0.0,0.0,0.0,11.0,Subcontractor,0.0,1957.0,0.0,0.0,0.0,9.0,250.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.000000,71.0,30.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.000000,NaN,1,0,0,0,0
116,5720651.0,2015,SA DES ETS RENE DUFOUR,1.51,1.00,NaN,5720651,6820B,31/12/2016,84.0,Auvergne-Rhône-Alpes,0.0,1,6820.0,,28.54,140.0,259.0,4664.0,SA DES ETS RENE DUFOUR,6820B,69382,01/01/2014,NaN,0.0,69.0,6820.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,69.0,131.0,0.0,0.00,0.0,01/01/1957,67.0,0.0,0.0,0.0,0.0,Subcontractor,0.0,1957.0,0.0,0.0,0.0,9.0,250.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,71.0,109.0,0.0,0.0,0.0,0.0,0.0,-38.0,0.0,0.0,0.0,0.000000,0.000000,0,1,0,0,0
117,5720651.0,2016,SA DES ETS RENE DUFOUR,1.17,0.94,17.14,5720651,6820B,31/12/2016,84.0,Auvergne-Rhône-Alpes,0.0,1,6820.0,,NaN,56.0,259.0,4664.0,SA DES ETS RENE DUFOUR,6820B,69382,01/01/2014,NaN,0.0,69.0,6820.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,69.0,48.0,1.0,-166.01,0.0,01/01/1957,67.0,0.0,0.0,0.0,135.0,Subcontractor,0.0,1957.0,0.0,0.0,0.0,9.0,250.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,138.0,0.000000,167.0,288.0,0.0,0.0,0.0,0.0,0.0,-121.0,0.0,0.0,0.0,0.000000,0.000000,0,0,1,0,0
118,5720651.0,2017,SA DES ETS RENE DUFOUR,NaN,NaN,NaN,5720651,6820B,31/12/2016,84.0,Auvergne-Rhône-Alpes,0.0,1,6820.0,,NaN,NaN,NaN,4664.0,SA DES ETS RENE DUFOUR,6820B,69382,01/01/2014,NaN,NaN,69.0,6820.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,69.0,NaN,NaN,NaN,NaN,01/01/1957,NaN,NaN,NaN,NaN,NaN,Subcontractor,NaN,1957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0,1,0
119,5720651.0,2018,SA DES ETS RENE DUFOUR,NaN,NaN,NaN,5720651,6820B,31/12/2016,84.0,Auvergne-Rhône-Alpes,0.0,1,6820.0,,NaN,NaN,NaN,4664.0,SA DES ETS RENE DUFOUR,6820B,69382,01/01/2014,NaN,NaN,69.0,6820.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,69.0,NaN,NaN,NaN,NaN,01/01/1957,NaN,NaN,NaN,NaN,NaN,Subcontractor,NaN,1957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
620,15450232.0,2014,T21,1.46,0.92,11.86,15450232,7022Z,31/12/2017,84.0,Auvergne-Rhône-Alpes,0.0,1,7022.0,,95.01,700.0,819.0,3556.0,T21,7022Z,69271,30/09/2016,NaN,0.0,69.0,7022.0,0.0,0.0,0.0,0.0,1.0,0.0,0

In [5]:

# Calculate the number of missing values in each column
missing_values = filtered_df.isnull().sum()

# If you're interested in columns with at least one missing value, you can filter those out:
columns_with_missing_values = missing_values[missing_values > 0]
print(columns_with_missing_values)


columns_wo_missing_values = missing_values[missing_values == 0]
print(columns_wo_missing_values)


captem_fa                     88061
wc_currentas                  68512
currentratio                  71510
regioncode                       70
nacerev2primarycodecode         145
solvencyratio                 73903
totalassets                   78931
totalfixedassets              94621
effectifmoyendupersonnel     173988
brevet                       117071
nacerév2codeprincipalcode       155
roa                          100748
currentas                     85109
creditors                     66043
financialdebtequity           66048
netsales                      79609
sharecapital                  66013
convertibleloans              95367
otherdebentureloans           95362
bankborrowings                66060
otherborrowings               95319
NetIntangible                 66269
tradedebts                    83198
marketable_sec                83241
cashNbank                     92043
land                         107729
buildings                    107720
plantAequip                 

In [ ]:
# Setting panel index
df = filtered_df.set_index(['siren', 'year'])

# Simple panel data description
df.describe()


,captem_fa,wc_currentas,currentratio,regioncode,numberofbranches,noofshareholders,nacerev2primarycodecode,solvencyratio,totalassets,totalfixedassets,ÿþmarquée,effectifmoyendupersonnel,brevet,départementcode,nacerév2codeprincipalcode,HT,MHT,MLT,LT,KIS,HTKIS,FKIS,OKIS,KIS_all,HT_all,man,trade,elec_eau,agr,trans,ict,real,cons,services,roa,deprtcode,currentas,creditors,financialdebtequity,netsales,sharecapital,convertibleloans,otherdebentureloans,bankborrowings,otherborrowings,NetIntangible,year_incorporation,tradedebts,marketable_sec,cashNbank,land,buildings,plantAequip,other_tangible,nettangible,fincharges,purch_good,purch_raw,wage_salaries,PropPlEq_inPRG,prepay_tangibleAs,conso_insvestm,customerprepay,taxes,debttosupplier_tangAs,otherdebts,totalDebt,InterestR,totalrevenues,totalexpenses,rawmaterials,workinprogr,ServINprogr,semifinNfinGoods,goodforsales,netincome,inventory,goodwill,othIntangAs,ratio,ratio_lag,Dum14,Dum15,Dum16,Dum17,Dum18
count,111254.000000,130803.000000,127805.000000,199245.000000,199315.000000,199315.000000,199170.000000,125412.000000,1.203840e+05,1.046940e+05,199315.000000,25327.000000,82244.000000,199315.0,199160.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000,98567.000000,199315.0,1.142060e+05,1.332720e+05,1.332670e+05,1.197060e+05,1.333020e+05,103948.000000,1.039530e+05,1.332550e+05,1.039960e+05,133046.000000,199315.000000,1.161170e+05,116074.000000,107272.000000,91586.000000,9.159500e+04,9.160000e+04,1.161150e+05,1.334780e+05,119862.000000,1.198340e+05,1.198390e+05,1.198670e+05,91590.000000,91579.000000,91576.000000,133400.000000,104125.000000,104086.000000,133436.000000,1.039390e+05,91352.000000,1.213070e+05,1.213080e+05,116078.000000,91583.000000,91584.000000,9.158400e+04,1.160780e+05,1.213060e+05,9.157400e+04,116082.000000,116079.000000,97779.000000,89697.000000,199315.000000,199315.000000,199315.000000,199315.000000,199315.000000
mean,32.170607,-7.632235,4.376215,83.965595,0.440383,0.647167,5824.754406,41.144219,5.938270e+03,4.592550e+03,9955.526453,49.042761,157.002456,69.0,5823.150884,0.002107,0.009959,0.026616,0.031056,0.176229,0.041868,0.111181,0.051878,0.381155,0.012066,0.070792,0.195520,0.008529,0.002358,0.025011,0.045130,0.069563,0.113413,0.803753,6.056889,69.0,2.760314e+03,5.851063e+02,2.967686e+02,4.313170e+03,1.124327e+03,80.650816,8.728400e+01,6.113431e+02,1.102049e+03,296.919111,2004.641999,8.496564e+02,165.833890,455.696799,102.939980,8.527103e+02,7.882501e+02,4.336069e+02,8.413330e+02,107.599489,1.062802e+03,6.471667e+02,7.264932e+02,83.152648,7.720798,2.169466,32.901949,417.914641,39.740244,336.733767,3.601037e+03,0.183779,4.865083e+03,4.555685e+03,83.099071,63.633764,26.672432,1.661092e+02,2.002795e+02,3.093606e+02,5.885802e+02,285.263366,62.791874,0.236658,0.236306,0.200000,0.200000,0.200000,0.200000,0.200000
std,3344.197454,844.147179,9.626987,1.522698,5.910667,1.477684,1731.626814,32.425404,7.495717e+04,7.009415e+04,5792.782869,705.922866,3917.458044,0.0,1731.857129,0.045856,0.099297,0.160959,0.173471,0.381015,0.200289,0.314357,0.221781,0.485672,0.109182,0.256479,0.396601,0.091959,0.048503,0.156158,0.207589,0.254410,0.317098,0.397159,76.203683,0.0,2.963570e+04,7.850241e+03,1.932403e+04,5.299689e+04,1.893276e+04,5440.962457,8.401168e+03,1.365318e+04,2.115303e+04,6347.481041,12.222005,9.747829e+03,4750.345052,5444.369801,2228.718521,2.541176e+04,1.461929e+04,1.327060e+04,2.692249e+04,2375.576430,1.462227e+04,1.946408e+04,2.062954e+04,2412.469716,752.328560,311.279164,700.325507,6910.149615,2535.556075,7458.711419,3.925084e+04,16.233322,5.933298e+04,5.621551e+04,1590.848884,1639.288580,1245.733825,1.039510e+04,5.647076e+03,8.412691e+03,1.318999e+04,6292.815131,1526.328486,0.238984,0.239285,0.400001,0.400001,0.400001,0.400001,0.400001
min,0.000000,-230710.000000,0.